# Importação das bibliotecas necessárias

In [1]:
import sys
import os
sys.path.append('/home/jovyan/work')

from utils.AlinharETL import AlinharETL
from pyspark.sql import types as T

# Crie uma instância da classe AlinharETL

In [2]:
# Crie uma instância da classe
bucket = "bronze"
datamart = "dadosabertosreceita"
extrator_bronze = AlinharETL(bucket,datamart)

2024-10-15 07:42:08,472 - INFO - Iniciando Sessão Spark.


# Parâmetros para leitura do arquivo

In [3]:
schema = T.StructType([
    T.StructField("cnpj_basico", T.StringType(), True),
    T.StructField("cnpj_ordem", T.StringType(), True),
    T.StructField("cnpj_dv", T.StringType(), True),
    T.StructField("id_matriz_filial", T.StringType(), True),
    T.StructField("nome_fantasia", T.StringType(), True),
    T.StructField("situacao_cadastral", T.StringType(), True),
    T.StructField("data_situacao_cadastral", T.StringType(), True),
    T.StructField("motivo_situacao_cadastral", T.StringType(), True),
    T.StructField("nome_cidade_exterior", T.StringType(), True),
    T.StructField("pais", T.StringType(), True),
    T.StructField("data_inicio_atividade", T.StringType(), True),
    T.StructField("cnae_fiscal_principal", T.StringType(), True),
    T.StructField("cnae_fiscal_secundaria", T.StringType(), True),
    T.StructField("tipo_logradouro", T.StringType(), True),
    T.StructField("logradouro", T.StringType(), True),
    T.StructField("numero", T.StringType(), True),
    T.StructField("complemento", T.StringType(), True),
    T.StructField("bairro", T.StringType(), True),
    T.StructField("cep", T.StringType(), True),
    T.StructField("uf", T.StringType(), True),
    T.StructField("municipio", T.StringType(), True),
    T.StructField("ddd_1", T.StringType(), True),
    T.StructField("telefone_1", T.StringType(), True),
    T.StructField("ddd_2", T.StringType(), True),
    T.StructField("telefone_2", T.StringType(), True),
    T.StructField("ddd_fax", T.StringType(), True),
    T.StructField("fax", T.StringType(), True),
    T.StructField("correio_eletronico", T.StringType(), True),
    T.StructField("situacao_especial", T.StringType(), True),
    T.StructField("data_situacao_especial", T.StringType(), True),
])

In [4]:
ano_mes = extrator_bronze.encontrar_ano_mes("dadosabertosreceita")
local = f"s3a://landing/dadosabertosreceita/{ano_mes}/estabelecimentos/*.csv"

In [5]:
config_adicional = {
    'sep': ';',
    'encoding': 'latin1'
}

# Leitura do arquivo

In [6]:
df_compliance_receita_estabelecimentos  = extrator_bronze.criar_view_temporaria_arquivo(local, 'compliance_estabelecimentos',schema,**config_adicional)

2024-10-15 07:42:12,647 - INFO - Lista de yaml carregada com sucesso
2024-10-15 07:42:16,254 - INFO - Criando view temporaria.


# Gravação no datalake

In [7]:
extrator_bronze.caminho_tabela_delta = 's3a://bronze/dadosabertosreceita/estabelecimentos'

In [8]:
extrator_bronze.salvar_delta(df_compliance_receita_estabelecimentos, 'overwrite')

2024-10-15 07:42:16,267 - INFO - Aguarde... Persistindo dados (overwrite)
2024-10-15 07:44:16,278 - INFO - Dados persistidos com sucesso
2024-10-15 07:44:16,280 - INFO - s3a://bronze/dadosabertosreceita/estabelecimentos
2024-10-15 07:44:16,281 - INFO - Aguarde... Realizando optimize
2024-10-15 07:48:39,587 - INFO - Optimize executado com sucesso.
2024-10-15 07:48:39,590 - INFO - Aguarde... Realizando vacuum
2024-10-15 07:48:57,987 - INFO - Vacuum executado com sucesso.


# Encerra sessão spark

In [9]:
extrator_bronze.parar_sessao()

2024-10-15 07:48:58,352 - INFO - Sessão Spark finalizada.
